# **Exploratory Data Analysis (EDA) with cleaned Ames Housing dataset**

## Objectives

* Read cleaned_data.csv from and confirm the dataset shape, column structure, and absence of missing values.
* Use appropriate libraries for analysis and visualisation:
   - `pandas` → data manipulation  
   - `numpy` → numerical operations  
   - `matplotlib.pyplot` → plotting distributions and relationships  
   - `seaborn` → advanced visualisations (correlation heatmap, boxplots, scatterplots)
* Explore the distribution of important numerical and categorical features.
* Investigate how different house attributes relate to `SalePrice`.
* Visualise correlations to identify the strongest predictor variables.
* Compare the client's inherited houses to the overall Ames market.
* Use the insights gained to guide feature selection for the modelling notebook.


## Inputs

* Write here which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Section 1

Section 1 content

---

# Section 2

Section 2 content

---